In [3]:
import cv2
import torch
import numpy as np
import torch.nn.functional as F
from torchvision import transforms
from models import PerformanceModel
from emotionoverlay import EmotionOverlay  # Import EmotionOverlay

In [1]:
from gifoverlay import GifEmotionOverlay


In [2]:
gif_overlay = GifEmotionOverlay("EmojiGif/")  # Use the correct folder


In [4]:
# Load the trained PyTorch model (Use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = "model/ferplus_model_pd_acc.pth"
model = PerformanceModel(input_shape=(1, 48, 48), n_classes=8, logits=True).to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

#Emotion categories (Order must match training data)
emotions = ["Neutral", "Happy", "Surprise", "Sad", "Angry", "Disgust", "Fear", "Contempt"]

#OpenCV face detector
haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
vid = cv2.VideoCapture(0)  # Start webcam

# Optimized Preprocessing Pipeline (No PIL conversion, Faster NumPy)
transform = transforms.Compose([
    transforms.ToTensor(),  
    transforms.Resize((48, 48)),  # Resize directly
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize
])




In [5]:
# Load the trained PyTorch model (Use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = "model/ferplus_model_pd_acc.pth"
model = PerformanceModel(input_shape=(1, 48, 48), n_classes=8, logits=True).to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

#Emotion categories (Order must match training data)
emotions = ["Neutral", "Happy", "Surprise", "Sad", "Angry", "Disgust", "Fear", "Contempt"]

#OpenCV face detector
haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
vid = cv2.VideoCapture(0)  # Start webcam

# Optimized Preprocessing Pipeline (No PIL conversion, Faster NumPy)
transform = transforms.Compose([
    transforms.ToTensor(),  
    transforms.Resize((48, 48)),  # Resize directly
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize
])




In [6]:
emotion_text_colors = {
    "Neutral": [(255,255,255), (224,212,196), (228,203,179)],
    "Happy": [(182,110,68), (76,235,253), (83,169,242)],
    "Surprise": [(247,255,0), (42,42,165), (232,206,0)],
    "Sad": [(194,105,3), (228,172,32), (237,202,162)],
    "Angry": [(61, 57, 242), (49,121,249), (232,220,214)],
    "Disgust": [(70,190,77), (120,159,6), (100,55,124)],
    "Fear": [(198, 128, 134), (133,71,68), (80,45,98)],
    "Contempt": [(160, 134, 72), (145, 180, 250), (173, 217, 251)]
}
color_index = 0  # Track which color is active


In [7]:
emotion_colors = {
    "Neutral": (255, 255, 255),  # White
    "Happy": (0, 255, 255),  # Yellow
    "Surprise": (0, 165, 255),  # Orange
    "Sad": (255, 0, 0),  # Blue
    "Angry": (0, 0, 255),  # Red
    "Disgust": (128, 0, 128),  # Purple
    "Fear": (255, 255, 0),  # Cyan
    "Contempt": (0, 255, 0)  # Green
}